In [1]:
import os
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models

# --- Config ---
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 64
DATASET_DIR = "AdversarialTestSets"
MODEL = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1).to(DEVICE).eval()

# --- Evaluation Function ---
@torch.no_grad()
def evaluate(model, loader, ks=(1, 5)):
    model.eval()
    correct = {k: 0 for k in ks}
    total = 0
    for x, y in loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        logits = model(x)
        _, pred = logits.topk(max(ks), dim=1)
        for k in ks:
            correct[k] += (pred[:, :k] == y.unsqueeze(1)).any(dim=1).sum().item()
        total += y.size(0)
    return {k: correct[k] / total for k in ks}

# --- Scan and Evaluate All .pt Files in Dataset Directory ---
print("\nTask 5: Transferability Evaluation on DenseNet121")
results = {}

for file in sorted(os.listdir(DATASET_DIR)):
    if file.endswith(".pt") and file.startswith("adv_test_set"):
        path = os.path.join(DATASET_DIR, file)
        print(f"\n Evaluating: {file}")
        data = torch.load(path)

        # Support both tuple and dict formats
        if isinstance(data, tuple):
            images, labels = data
        else:
            images = data["images"]
            labels = data["labels"]

        loader = DataLoader(TensorDataset(images, labels), batch_size=BATCH_SIZE, shuffle=False)
        acc = evaluate(MODEL, loader)
        print(f"Top-1 Accuracy: {acc[1]*100:.2f}% | Top-5 Accuracy: {acc[5]*100:.2f}%")
        results[file] = {"top1": acc[1]*100, "top5": acc[5]*100}

# --- Optional: Print Summary ---
print("\nSummary of Transferability (DenseNet121):")
for fname, acc in results.items():
    print(f"{fname:<35}  Top-1: {acc['top1']:.2f}%  |  Top-5: {acc['top5']:.2f}%")



Task 5: Transferability Evaluation on DenseNet121

 Evaluating: adv_test_set1.pt
Top-1 Accuracy: 63.40% | Top-5 Accuracy: 89.40%

 Evaluating: adv_test_set2.pt
Top-1 Accuracy: 63.20% | Top-5 Accuracy: 91.00%

 Evaluating: adv_test_set3_C1.pt
Top-1 Accuracy: 64.40% | Top-5 Accuracy: 87.20%

 Evaluating: adv_test_set3_untargeted.pt
Top-1 Accuracy: 70.60% | Top-5 Accuracy: 92.20%

Summary of Transferability (DenseNet121):
adv_test_set1.pt                     Top-1: 63.40%  |  Top-5: 89.40%
adv_test_set2.pt                     Top-1: 63.20%  |  Top-5: 91.00%
adv_test_set3_C1.pt                  Top-1: 64.40%  |  Top-5: 87.20%
adv_test_set3_untargeted.pt          Top-1: 70.60%  |  Top-5: 92.20%


In [3]:
import os
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models

# --- Config ---
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 64
DATASET_DIR = "AdversarialTestSets"

# --- Load ResNet18 Model ---
resnet18 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1).to(DEVICE).eval()

# --- Evaluation Function ---
@torch.no_grad()
def evaluate(model, loader, ks=(1, 5)):
    model.eval()
    correct = {k: 0 for k in ks}
    total = 0
    for x, y in loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        logits = model(x)
        _, pred = logits.topk(max(ks), dim=1)
        for k in ks:
            correct[k] += (pred[:, :k] == y.unsqueeze(1)).any(dim=1).sum().item()
        total += y.size(0)
    return {k: correct[k] / total for k in ks}

# --- Main Evaluation Loop ---
results = {}

for file in sorted(os.listdir(DATASET_DIR)):
    if file.endswith(".pt") and file.startswith("adv_test_set"):
        path = os.path.join(DATASET_DIR, file)
        print(f"\n Evaluating: {file}")

        data = torch.load(path)
        if isinstance(data, tuple):
            images, labels = data
        else:
            images = data["images"]
            labels = data["labels"]

        loader = DataLoader(TensorDataset(images, labels), batch_size=BATCH_SIZE, shuffle=False)
        acc = evaluate(resnet18, loader)
        results[file] = {"top1": acc[1]*100, "top5": acc[5]*100}
        print(f"🧪 ResNet18 → Top-1: {acc[1]*100:.2f}% | Top-5: {acc[5]*100:.2f}%")

# --- Summary Output ---
print("\n ResNet18 Transferability Summary:")
print(f"{'Dataset':<35} {'Top-1 (%)':>10} {'Top-5 (%)':>10}")
print("-" * 60)
for fname, acc in results.items():
    print(f"{fname:<35} {acc['top1']:>10.2f} {acc['top5']:>10.2f}")



 Evaluating: adv_test_set1.pt
🧪 ResNet18 → Top-1: 58.40% | Top-5: 82.80%

 Evaluating: adv_test_set2.pt
🧪 ResNet18 → Top-1: 57.20% | Top-5: 83.80%

 Evaluating: adv_test_set3_C1.pt
🧪 ResNet18 → Top-1: 61.20% | Top-5: 82.60%

 Evaluating: adv_test_set3_untargeted.pt
🧪 ResNet18 → Top-1: 64.80% | Top-5: 89.00%

 ResNet18 Transferability Summary:
Dataset                              Top-1 (%)  Top-5 (%)
------------------------------------------------------------
adv_test_set1.pt                         58.40      82.80
adv_test_set2.pt                         57.20      83.80
adv_test_set3_C1.pt                      61.20      82.60
adv_test_set3_untargeted.pt              64.80      89.00
